In [ ]:
#libraries
import os
os.chdir('analysis/multi_freq_from_archive')

from astroquery.alma import Alma
alma = Alma()
from astropy import units as u
from astropy import coordinates
import numpy as np
from astroquery.nvas import Nvas
import requests
import shutil
import os

from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import skycoord_to_pixel

import os
from pathlib import Path
import sys

import wget

import warnings
from astropy.utils.exceptions import AstropyWarning

from collections import Counter

warnings.simplefilter('ignore', category=AstropyWarning)

In [ ]:
#Path on local computer to SMA Images
parent_dir = Path.cwd().parent
smapath=f'{parent_dir}/Gaussian_Fit_Pipline/230316_128/images/flag1_cal1/fields/science'

In [ ]:
#functions to obtain coordinates from sourcename
from astroquery.ipac.ned import Ned
from PyAstronomy import pyasl
from astropy import coordinates
import astropy.units as u
import numpy as np
import re

def getname(coord2,sourcename):
    result_table = Ned.query_region(coord2, radius=1*u.arcsec, equinox='J2000')    
    allnames=np.unique(result_table['Object Name'])
    newsourcename='NA'
    if len(allnames)>1:
        while 0==0:
            input(f'{len(allnames)} names exist for this location.  When cycling through names press y to store name.  Press Enter to start the cycling process.')
            keepnames=[]
            for iall in allnames:
                yn=input(f'{iall}')
                if yn=='y':
                    isourcename=iall
                    yn=input(f'The name you have choosen is {isourcename}.  Enter CHOOSE for this naming priority.  Enter KEEP to keep this name, but not as a priority.')

                    if ' ' in isourcename:
                        isourcename=isourcename.replace(' ','')
                    if '\n' in isourcename:
                        isourcename=isourcename.replace('\n','')  

                    if yn=='CHOOSE':
                        newsourcename=isourcename
                    if yn=='KEEP':
                        keepnames.append(isourcename)   

            if newsourcename!='NA':
                sourcename=newsourcename

            input(f'your have chosen {sourcename} for the priority name')
            input(f'your have chosen {keepnames} for the secondary names')
            numchoose=input('Enter "AGAIN" to see the list again, or Enter to continue.')
            if numchoose=='AGAIN':
                continue
            else:
                break
    else:
        newname=allnames[0]
        if ' ' in newname:
            newname=newname.replace(' ','')
        if '\n' in newname:
            newname=newname.replace('\n','')  
        if newname.upper()==sourcename.upper():
            sourcename=allnames[0]
            keepnames=['NA']
        else:
            keepnames=[newname]
    return(sourcename,keepnames)

def getsourcenames():
    files=[]
    txtsources='sources.txt'

    if os.path.exists(txtsources):
        with open(txtsources, 'r') as infile:
            remembernames=[]
            for params in infile:
                sourcename=params
                try:
                    result_table = Ned.query_object(sourcename)
                except Exception as e:
                    sourcename=input('Sourcename does not seem to exist.  Enter "1" to enter coordinates or rewrite "sources.txt"')

                    if sourcename=='1':
                        coord=input(f'Enter coordinates in J2000 hh:mm:ss dd:mm:ss')
                        if ' ' not in coord:
                            input("Improper coordinate format.  Try again")
                            continue
                        coord=coord.split(' ')
                        ra=coord[0]
                        dec=coord[1]
                        try:
                            coord2 = coordinates.SkyCoord(ra=ra, dec=dec, unit=(u.hourangle, u.deg), frame='fk5')
                        except Exception as e:
                            input('Improper Coordinates.  Try again.')
                            continue
                        sourcename,keepnames=getname(coord2,sourcename)  
                        input(f'The priority name of the source you have chosen is {sourcename}')
                        result_table = Ned.query_object(sourcename)

                ra=result_table['RA'][0]
                dec=result_table['DEC'][0]
                sexa = pyasl.coordsDegToSexa(ra, dec)
                sexa=sexa.split(' ')
                #rasexa=f'{sexa[0]}h{sexa[1]}m{sexa[2]}s'
                #decsexa=f'{sexa[0]}d{sexa[1]}m{sexa[2]}s'
                rasexa=f'{sexa[0]}:{sexa[1]}:{sexa[2]}'
                decsexa=f'{sexa[4]}:{sexa[5]}:{sexa[6]}'

                coord2 = coordinates.SkyCoord(ra=rasexa, dec=decsexa, unit=(u.hourangle, u.deg), frame='fk5')
                
                if ' ' in sourcename:
                    sourcename=sourcename.replace(' ','')
                if '\n' in sourcename:
                    sourcename=sourcename.replace('\n','')

                if sourcename!='NGC5765b':
                    pass
                    #continue

                sourcename,keepnames=getname(coord2,sourcename) 

                if ' ' in sourcename:
                    sourcename=sourcename.replace(' ','')
                if '\n' in sourcename:
                    sourcename=sourcename.replace('\n','')

                totconts=[]
                sciname1=sourcename
                sciname2=keepnames

                # Find all sequences of digits in the string
                number_sequences1 = re.findall(r'\d+', sciname1)

                if number_sequences1!=[]:

                    # Find the maximum length of the sequences
                    max_length1 = max(len(seq) for seq in number_sequences1)

                    # Get all sequences that have the maximum length
                    longest_sequences1 = [seq for seq in number_sequences1 if len(seq) == max_length1]

                else:
                    newfind=1
                    for iremembernames in remembernames:
                        if iremembernames[0]==sciname1:
                            newfind=0
                            sciname1=iremembernames[1]
                    if newfind==1:
                        sciname1a=input(f"Enter a name to look for in file names for {sciname1}")
                        remembernames.append([sciname1,sciname1a])
                        sciname1=sciname1a

                for itotconts in longest_sequences1:
                    totconts.append(itotconts)

                for isciname2 in sciname2:
                    if sciname2[0]=='NA':
                        continue
                    number_sequences2 = re.findall(r'\d+', isciname2)
                    max_length2 = max(len(seq) for seq in number_sequences2)
                    longest_sequences2 = [seq for seq in number_sequences2 if len(seq) == max_length2]

                    for itotconts2 in longest_sequences2:
                        totconts.append(itotconts2)

                file=[[sourcename,keepnames],[totconts,rasexa,decsexa]]
                files.append(file)
    else:
        input('Create file for source names names "sources.txt" with one line per source.')
    return(files)

In [ ]:
#make filenames using functions
import os
from pathlib import Path
import sys

txtnamesandcoords='namesandcoords.txt'

yn2='y'
if os.path.exists(txtnamesandcoords):
    with open(txtnamesandcoords, 'r') as infile:
        for params in infile:
            params=eval(params)
            for inamecoords in params:
                oname=inamecoords[0]
                nname=inamecoords[1][0]
                ra=inamecoords[1][1]
                dec=inamecoords[1][2]
                #print(f'{oname.ljust(30)}: {nname}: {ra} {dec}')
                print(oname)
                files=inamecoords
    namesandcoords=params
    input(f'{txtnamesandcoords} already exist.  Take a look.')
    yn2=input('Do you want to erase and write another? y/n')

In [ ]:
#string killer
def killstrings(kills,file):
    rasexa=file[1]
    decsexa=file[2]

    allkills=[]
    for ikill in kills:
        allkills.append(ikill)

    thisnamer=file[0]
    thisnamer3='notstring'
    newx=0

    killtxt='killtxt.txt'

    if os.path.exists(killtxt):
        with open(killtxt, 'r') as infile:
            for params in infile:
                allkills.append(params)

    for iall in allkills:
        if newx==0:
            thisnamerar=[thisnamer]
            thisnamer2=thisnamerar.copy()
            thisnamer2=thisnamer2[0]
        if newx==1:
            thisnamer2=thisnamer3
        kill1=[]
        kill2=[]
        if iall in thisnamer2:
            kill1=thisnamer2.split(iall)[0]
            kill2=thisnamer2.split(iall)[1]
        if len(kill1)!=0:
            if len(kill2)!=0:
                break
        if len(kill1)!=0:
            if len(kill2)==0:
                thisnamer3=kill1
                newx=1
        if len(kill2)!=0:
            if len(kill1)==0:
                thisnamer3=kill2 
                newx=1
    try: 
        for i in thisnamer3:
            int(i)
        filename=thisnamer3
        return([[filename,rasexa,decsexa],allkills])
    except Exception as e:
        pass


    while 0==0:     
        if thisnamer3!='notstring':
            thisnamer=thisnamer3      
        if thisnamer3=='notstring':
            thisnamer=file[0]   
        kill=input(f'{thisnamer} choose a string to kill.  Enter 1 to choose your own file name.  Enter 2 to print off the string to copy and paste.  Press 3 to keep as is.')
        if kill=='2':
            print(thisnamer)
            kill=input(f'{thisnamer} choose a string to kill.  Enter 1 to choose your own file name.')
        if kill=='':
            yn=input(f'Do you want to keep source name "{thisnamer}" as is?  Enter y/n')
            if yn=='y':
                filename=thisnamer
                return([[filename,rasexa,decsexa],allkills])
            else:
                continue
        if kill=='1':
            newname=input(f'Enter New Filename')
            yn=input(f'Do you want to keep source name "{newname}"?  Enter y/n')
            if yn=='y':
                filename=newname
                return([[filename,rasexa,decsexa],allkills])
            if yn=='n':
                continue
        if kill=='3':
            filename=thisnamer
            return([[filename,rasexa,decsexa],allkills])
        killname=thisnamer.split(kill)
        if len(killname)==1:
            input('String you Entered is not in source try again')
            continue

        allkills.append(kill)
        kill1=thisnamer.split(kill)[0]
        kill2=thisnamer.split(kill)[1]

        borf='3'
        if len(kill1)!=0:
            if len(kill2)!=0:
                input('Enter a string to kill that is not in the middle')
                continue

        if len(kill1)!=0:
            if len(kill2)==0:
                filename=kill1
                borf='4'

        if len(kill2)!=0:
            if len(kill1)==0:
                filename=kill2
                borf='4'

        if borf==3:
            input('something is wrong')

        yn=input(f'Do you want to keep source name "{filename}"?  Enter y/n')
        if yn=='y':
            return([[filename,rasexa,decsexa],allkills])
        if yn=='n':
            yn=input(f'Do you want to remove more strings from "{filename}"?  Enter y.  Otherwise you will restart the naming process.')
            if yn=='y':
                newsend=([filename,rasexa,decsexa])
                new=killstrings(allkills,newsend)
                return([[new[0][0],rasexa,decsexa],new[1]])
            continue

In [ ]:
#write new strings with coordinates to a text
if yn2=='y': 
    nfiles=getsourcenames()
    txtnamesandcoords='namesandcoords.txt'
    if os.path.exists(txtnamesandcoords):
        os.remove(txtnamesandcoords)
    with open(txtnamesandcoords, 'w') as file:
        original_stdout = sys.stdout
        sys.stdout = file
        print(nfiles)
        sys.stdout = original_stdout

In [ ]:
#collect urls from ALMA and VLA for selected targets
#iterating for sources

allsource_urls=[]

if os.path.exists(txtnamesandcoords):
    with open(txtnamesandcoords, 'r') as infile:
        for params in infile:
            params=eval(params)
            files=params

try:
    VLAlist=os.listdir('VLA')
    ALMAlist=os.listdir('ALMA')
    checksize=len(VLAlist)+len(ALMAlist)
except Exception as e:
    checksize=0

if checksize!=0:
    downloadyn=input('Do you want to redownload urls?  y/n  ')

if downloadyn=='y':
    print('The first round of filtering only keeps files that contain ".fit"')
    print('And filters out names with "_ph", "_bp", "README", fluxcal", "phasecal\n')
    for file in files:
        source=file[0][0]

        #urls by a single source
        all_urls=[]
        ra=file[1][1]
        dec=file[1][2]
        galactic_center = coordinates.SkyCoord(ra, dec, frame='icrs', unit=(u.hourangle, u.deg))
        print(f'vla urls for {source}')
        vlaobs = Nvas.get_image_list(galactic_center, radius=2*u.arcsec)

        #vla by observation
        print(f'vla sizes for {source}')
        for i in vlaobs:
            url = i
            if not 'fits' in url:
                print(url)
                input('check url')
            response = requests.head(url)
            file_size = int(response.headers.get("Content-Length", 0))

            all_urls.append([source,'vla',url,file_size])
        print(f'alma observations for {source}')
        gc_data = alma.query_region(galactic_center, 2*u.arcsec)

        if len(gc_data)!=0:
            membernames=[]
            for igc_data in gc_data:
                membername=igc_data['member_ous_uid']
                membernames.append(membername)
            membernames= np.unique(membernames)

            #alma by observation
            for membername in membernames:    
                print(f'alma urls for {source} : {membername}')
                almaobs = alma.get_data_info(membername, expand_tarfiles=True)

                allboth=[]
                for i in almaobs:
                    url=i['access_url']
                    uid=i['ID']
                    size=i['content_length']

                    #response = requests.head(url)
                    #file_size = int(response.headers.get("Content-Length", 0))
                    #print(size)
                    #print(file_size)
                    #input('check')

                    both=[url,uid,size]
                    allboth.append(both)
                #eliminating phase and bandpass urls
                all_urls_3=[]
                for i in allboth:
                    url = i[0]
                    if '_ph' in url:
                        continue
                    if '_bp' in url:
                        continue
                    if 'README' in url:
                        continue
                    if 'fluxcal' in url:
                        continue
                    if 'phasecal' in url:
                        continue
                    all_urls_3.append([source,'alma',i])

                #sorting a single observation by suffix
                tarfiles=[]
                fitfiles=[]
                xmlfiles=[]
                tgzfiles=[]
                txtfiles=[]
                pyfiles=[]
                logfiles=[]
                pdffiles=[]
                csvfiles=[]
                htmlfiles=[]
                datfiles=[]
                pngfiles=[]

                for i in all_urls_3:
                    url=i[2][0]
                    if '.tar' in url:
                        tarfiles.append(i)
                    elif '.fit' in url:
                        fitfiles.append(i)
                    elif '.xml' in url:
                        tgzfiles.append(i)
                    elif '.tgz' in url:
                        tgzfiles.append(i)
                    elif '.txt' in url:
                        tgzfiles.append(i)
                    elif '.py' in url:
                        pyfiles.append(i)
                    elif '.log' in url:
                        logfiles.append(i)
                    elif '.pdf' in url:
                        pdffiles.append(i)
                    elif '.csv' in url:
                        csvfiles.append(i)
                    elif '.html' in url:
                        htmlfiles.append(i)
                    elif '.dat' in url:
                        datfiles.append(i) 
                    elif '.png' in url:
                        pngfiles.append(i) 
                    else:
                        print(i)
                        input('check')

                #storing files with .fit from a single observation to the source url
                all_urls.append(fitfiles)
        if len(gc_data)!=0:
            all_urls.append([])
            
        allsource_urls.append(all_urls)


In [ ]:
#checking for repeated urls
if downloadyn=='y':
    adatacheck=[]
    for i in allsource_urls:
        for ii in i:
            if len(ii)==0:
                continue
            if ii[1]=='vla':
                pass
            elif ii[0][1]=='alma':
                for iii in ii:
                    adatacheck.append(iii)
            else:
                print(ii)
                input('check telescope')

    k1=-1
    for i in adatacheck:
        k1=k1+1
        k2=-1
        for ii in adatacheck:
            k2=k2+1
            if i[2][0]==ii[2][0]:
                if k2!=k1:
                    print(i)
                    input('check')

In [ ]:
#sort through urls to find continuum images
if downloadyn=='y':
    vla_obs=[]
    conts=[]

    readmes=[]
    contsub=[]
    tt0s=[]
    tt1s=[]
    alphas=[]
    lines=[]

    mfss=[]
    cubes=[]
    repBWs=[]
    sds=[]
    concats=[]

    Windows=[]
    SBrings=[]

    splitcals=[]
    manuals=[]

    chemicalurls=[]
    others=[]

    chemicals=['SiO',
            'HeII',
            'CS',
            'H13CN',
            'HCN',
            'CO6',
            '13CO',
            'CH3CN',
            'CH3OH',
            'HC3N',
            'C18O',
            'CO32',
            'CS76',
            'HCOp43',
            'C17O',
            'CN',
            'H2CO',
            'HCN',
            'NHC',
            'C3H2',
            'SO',
            'CO21',
            'CO',
            'HCOp',
            'H20',
            'HNC',
            'HC15N',
            'co32',
            ]

    print('These observations do not have the following words in the filename: "cont", "mfs", "repBW", "cube", "sd", "split.cal".  However These words may still supply a continuum image file')
    print('Files with the following words have also been filtered out: "readme", "contsub", "tt0", "tt1", "alpha", "line", "concats", and a variety of chemical names such as "SiO')
    print('***Look at the README for theses missing uids, as well as the observations that have words in the filename that may still supply a continuum image***')
    print('***Figure out which band and how many other images at what epoch and resolution these images are to asses the necissesity of producing a continuum image***\n')
    #https://almascience.nrao.edu/portal/documents-and-tools/cycle9/alma-qa2-data-products-for-cycle-9
    for i in allsource_urls:
        for ii in i:
            others=[]
            if len(ii)==0:
                continue
            if ii[1]=='vla':
                vla_obs.append(ii)
            if ii[0][1]=='alma':
                ireadmes=[]
                icontsub=[]
                itt0s=[]
                itt1s=[]
                ialphas=[]
                ilines=[]
                iconts=[]
                iconcats=[]

                imfss=[]
                icubes=[]
                irepBWs=[]
                iWindows=[]
                iSBrings=[]
                isds=[]

                isplitcals=[]
                imanuals=[]

                ichemicalurls=[]
                iothers=[]

                itot=[]
                contcheck=[]
                for iii in ii:
                    itot.append(iii)
                    size=iii[2][2]
                    uid=iii[2][1]
                    url=iii[2][0]
                    source=iii[0]
                    #Make a Function for Doing the following
                    if 'README' in url:
                        ireadmes.append(ii)
                    #continuum subtracted
                    elif 'contsub' in url:
                        icontsub.append(url)
                    #0th order Taylor Expansion
                    elif 'tt0' in url:
                        itt0s.append(url)
                    #First order T.E.
                    elif 'tt1' in url:
                        itt1s.append(url)
                    #Exponential Parameter
                    elif 'alpha' in url:
                        ialphas.append(url)
                    #Spectral Line
                    elif 'line' in url:
                        ilines.append(url)
                    #"aggregate bandwidth image"
                    #Single bandwdith with extra data?
                    elif 'mfs' in url:
                        imfss.append(url)
                    #2D Position and 1D Spectral => 3D Cube Array
                    elif 'cube' in url:
                        icubes.append(url)
                    #"cube at representative bandwidth"
                    elif 'repBW' in url:
                        irepBWs.append(url)
                    #Single Dish Observation
                    elif 'sd' in url:
                        isds.append(url)
                    #Several visibility data sets
                    elif 'concat' in url:
                        iconcats.append(url)

                    elif 'split.cal' in url:
                        isplitcals.append(url)          
                    
                    elif 'Window_a' in url:
                        iWindows.append([url,uid])

                    elif 'SBring' in url:
                        iSBrings.append([url,uid])

                    elif 'manual' in url:
                        imanuals.append([url,uid])

                    #Continuum
                    else:
                        otherchecker=0
                        for chem in chemicals:
                            if chem in url:
                                ichemicalurls.append(url)
                                otherchecker=1

                        if otherchecker==0:
                            if 'cont' in url:
                                iconts.append([url,size,source])
                                contcheck.append(url)
                            else:
                                iothers.append(url)

                Windows.append(iWindows)
                SBrings.append(iSBrings)
            
                if len(contcheck)==0:
                    if len(imfss)==0:
                        if len(icubes)==0:
                            if len(isds)==0:
                                if len(iconcats)==0:
                                    if len(ichemicalurls)==0:
                                        if len(isplitcals)==0:
                                            if len(imanuals)==0:
                                                for i in itot:
                                                    print(i)
                                                print('\n')
                conts.append(iconts)

    for i in Windows:
        for ii in i:
            url=ii[0]
            uid=ii[1]
            print(f'{uid}: Look at README')
            print(f'"_Window_a" in url')
            print(url)
            print('\n')


    for i in SBrings:
        for ii in i:
            url=ii[0]
            uid=ii[1]
            print(f'{uid} Look at the README for ')
            print(f'"SBring" in url:')
            print(url)
            print('\n')

In [ ]:
#filter through continuums to choose one per observation
import re

if downloadyn=='y':
    def keeponly_ifcan(contin,x):
        contout=[]
        xlower=x.lower()
        for contj in contin:
            contjlower=contj[0].lower()
            #capitalization will not matter
            if xlower in contjlower:
                contout.append(contj)
        if len(contout)==0:
            contout=contin

        return(contout)



    if os.path.exists(txtnamesandcoords):
        with open(txtnamesandcoords, 'r') as infile:
            for params in infile:
                params=eval(params)
                files=params    

    newvals=[]
    sucess=0

    remembernames=[]

    for i in conts:
        if i==[]:
            continue
        #https://almascience.eso.org/alma-data/ari_l
        #ari_l seemse to be a more upgraded preprocessing pipeline
        #pb_cor is the the cleaned imaged
        #regcal as used as apposed to selfcal
        source=i[0][2]
        tester=0
        for file in files:
            isource=file[0][0]
            if isource==source:
                totconts=file[1][0]
                tester=1
        if tester==0:
            input(f"something wrong with {source}")

        newcontsb1=[]
        for itotconts in totconts:
            newcontsb1.append(keeponly_ifcan(i,itotconts))
        newcontsb2=[]
        nofind=1
        for inewconts in newcontsb1:
            #as long as inewconts is not the same as i newcontsb2 is filled and set to newconts
            if len(inewconts)!=len(i):
                for i2 in inewconts:
                    newcontsb2.append(i2)
        if newcontsb2!=[]:
            newconts=newcontsb2
        else:
            #resorts to returning the original list
            newconts=keeponly_ifcan(i,source)


        newconts2=keeponly_ifcan(newconts,'pbcor')
        newconts3=keeponly_ifcan(newconts2,'ari_l')
        newconts4=keeponly_ifcan(newconts3,'regcal')

        if len(newconts4)==0:
            input('error')
        if len(newconts4)>1: 
            for inewconts4 in newconts4:
                print(inewconts4[0])
            print('\n')
            sucess=sucess+1

        newvals.append(newconts4)

    if sucess==0:
        print('SUCESS!  One continuum per observation If not Filtered Out')
    else:
        print('More Work to do.  More than one continuum per observation')

In [ ]:
#Download urls -- one per observation
if downloadyn=='y':
    path1='ALMA'
    if os.path.exists(path1):
        shutil.rmtree(path1)
    os.mkdir(path1)

    path2='VLA'
    if os.path.exists(path2):
        shutil.rmtree(path2)

    os.mkdir(path2)

    #for alma
    for inewvals in newvals:
        if len(inewvals)!=1:
            input(f'more than 1 file for {inewvals}')
        tele='alma'
        source=inewvals[0][2]
        size=inewvals[0][1]
        url=inewvals[0][0]

        path3=f'{path1}/{source}'
        if not os.path.exists(path3):
            os.mkdir(path3)

        path4=f'{path3}'

        print(f'{url} Downloading')
        wget.download(url, out=path4)
        print(f'{url} Done')

        matcher=0
        for ifile in files:
            if ifile[0][0]==source:
                ra=ifile[1][1]
                dec=ifile[1][2]
                matcher=1

        if matcher==0:
            input(f'no matchers for alma {url}')

    for ivlas in vla_obs:
        tele='vla'
        source=ivlas[0]
        size=ivlas[3]
        url=ivlas[2]

        path3=f'{path2}/{source}'
        if not os.path.exists(path3):
            os.mkdir(path3)

        path4=f'{path3}'
        #input(path4)
            
        print(f'{url} Downloading')
        wget.download(url, out=path4)
        print(f'{url} Done')

        matcher=0
        for ifile in files:
            if ifile[0][0]==source:
                ra=ifile[1][1]
                dec=ifile[1][2]
                matcher=1

        if matcher==0:
            input(f'no matchers for vla {url}')

In [ ]:
#Make sure the coordinate is in the image
if downloadyn=='y':
    def check_coordinates_in_image(file_path, ra_hms, dec_dms):
        try:
            # Convert RA/Dec from sexagesimal to decimal degrees
            target_coords = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
            #ra_decimal, dec_decimal = target_coords.ra.deg, target_coords.dec.deg
            # Open the file and extract WCS and image size
            with fits.open(file_path) as hdul:
                wcs = WCS(hdul[0].header)
                image_size = hdul[0].data.shape[-2:]  # Get spatial dimensions
            # Convert RA/Dec to pixel coordinates
            pixel_coords = skycoord_to_pixel(target_coords, wcs)
            # Check if pixel coordinates are within image bounds
            x, y = pixel_coords
            inside_image = (0 <= x < image_size[1]) and (0 <= y < image_size[0])
            return inside_image
        except Exception as e:
            return {"error": str(e)}
        
    #for icheck in allcheck:
    #file_path = 'VLA/4258/1.41I1.23_AD0364_1995AUG28_1_23.9U3.92M.imfits'
    #file_path = 'VLA/4258/1.45I5.42_AH782_2002AUG11_1_213.U3000M.imfits'

        for ifile in files:
            continue
            input(ifile[0])
            if ifile[0]==source:
                ra=ifile[1]
                dec=ifile[2]
                matcher=1
        if matcher==0:
            input(f'no matchers for {url}') 

    #file_path = 'VLA/5765b/1.45I5.42_AH782_2002AUG11_1_213.U3000M.imfits'
    #ra='14:50:51.519'
    #dec='05:06:52.317'

    #ra_hms = "12:18:57.5046"
    #dec_dms = "47:18:14.303"
    #files=icheck[0]
    #ra=icheck[1][0]

    path1='ALMA'
    path2='VLA'

    #alma

    whichsources=[]
    totsources=[]
    badfiles=[]

    almasources=[]
    for inewvals in newvals:
        source=inewvals[0][2]
        almasources.append(source)
    ualmasources = list(set(almasources))


    for source in ualmasources:

        #path1 is alma
        path3=f'{path1}/{source}'
        path4=f'{path3}'

        for ifile in files:
            if ifile[0]==source:
                ra=ifile[1]
                dec=ifile[2]
                matcher=1
        if matcher==0:
            input(f'no matchers for {url}')

        almafiles=os.listdir(path4)
        for ialmafile in almafiles:
            ialmafile=f'{path4}/{ialmafile}'
            if os.path.exists(ialmafile):
                totsources.append(source)
                cheval=check_coordinates_in_image(ialmafile,ra,dec)
                if cheval!=True:
                    print(ialmafile)
                    badfiles.append(ialmafile)
                    whichsources.append(source)
                    if cheval!=False:
                        input(f'not T or F:{cheval}')

    vlasources=[]
    for ivals in vla_obs:
        source=ivlas[0]
        vlasources.append(source)
    uvlasources = list(set(almasources))

    for source in uvlasources:
        source=ivlas[0]

        path3=f'{path2}/{source}'
        path4=f'{path3}'

        for ifile in files:
            if ifile[0]==source:
                ra=ifile[1]
                dec=ifile[2]
                matcher=1
        if matcher==0:
            input(f'no matchers for {url}')

        vlafiles=os.listdir(path4)
        for ivlafile in vlafiles:
            if os.path.exists(ivlafile):
                totsources.append(source)
                if cheval!=True:
                    print(ivlafile)
                    badfiles.append(ivlafile)
                    whichsources.append(source)
                    if cheval!=False:
                        input(f'not T or F:{cheval}')

In [ ]:
#Get Percentage of files that are outside of the coordinate for each source
if downloadyn=='download':
    conts = Counter(whichsources)
    contstot = Counter(totsources)

    key_to_dicts = defaultdict(list)

    for key in set(conts.keys()).union(contstot.keys()):
        key_to_dicts[key].append(conts.get(key, 0)) 
        key_to_dicts[key].append(contstot.get(key, 0))  

    for key, counts in key_to_dicts.items():
        pass
        #print(f"{key}: {counts}")

    percentages = {
        key: (counts[0] / counts[1]) * 100 if counts[1] != 0 else 0
        for key, counts in key_to_dicts.items()
    }

    print("Percentage of Files that are wrong coordinates for Each Source:")
    for key, percentage in percentages.items():
        if percentage!=0:
            print(f"{key}: {percentage:.2f}%")

In [ ]:
# Remove Files where coordinate is outside of image
if downloadyn=='download':
    for i in badfiles:
        print(f"Processing: {i}")
        if os.path.exists(i):
            try:
                if os.path.isfile(i):
                    os.remove(i)  # Remove file
                    print(f"File removed: {i}")
                elif os.path.isdir(i):
                    shutil.rmtree(i)  # Remove directory
                    print(f"Directory removed: {i}")
            except Exception as e:
                print(f"Error while removing {i}: {e}")
        else:
            print(f"Path does not exist: {i}")